In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os


In [8]:
from noise_support_codes import power, snr, noise_power_for_fixed_SNR_dB, find_noise_for_SNR, noise_analysis

In [9]:
import matplotlib.pyplot as plt
from Codes import (skew_tent, iterations, firingtime_calculation, probability_calculation, class_avg_distance, cosine_similar_measure, class_wise_data, test_split_generator,chaos_method, CHAOSNET)

In [10]:
from load_data import get_data
from parameterfile import parameter_file
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import logging

In [11]:
data_name = "IRIS"
method = "TT-SS"
var_num = 42

In [13]:
import os
folder_path = os.getcwd() + "/"+ "chaosnet-results/" + data_name + "/" + method + "/"+ data_name + "-test_proba.csv"
print(" Accessing Test Proba " , folder_path)
test_proba = np.array(pd.read_csv(folder_path, header = None))
try:
    assert np.min(test_proba) >= 0 and np.max(test_proba <= 1)
except AssertionError:
    logging.error("ERROR-Check test_proba.csv and check the codes properly", exc_info=True)


X_train, y_train, X_test, y_test = get_data(data_name, var_num)

a, b, c, q, length, num_classes, samples_per_class, check, details, var, method, epsilon = parameter_file(data_name)


row = -1
i = 6
print("Number of samples_per_class = ", i+1)
rep_path = os.getcwd() + "/"+ "chaosnet-results/" + data_name + "/" + method +"/"+  data_name + "-" + method + "-representation_vector_last" + str(i) + ".csv"
representation_vectors = np.array(pd.read_csv(rep_path, header = None))
SNR = []
Ps = power(representation_vectors)
length1 = representation_vectors.shape[0] * representation_vectors.shape[1]

#variance = np.arange(0.005, 0.816, 0.001 )
variance = np.arange(0.00011, 0.0456, 0.00004 )

ACC = np.zeros((len(variance), 1))
PRECISION = np.zeros((len(variance), 1))
RECALL = np.zeros((len(variance), 1))
F1SCORE = np.zeros((len(variance), 1))
for sd in variance:
    row = row + 1
    np.random.seed(42)
    noise = np.random.normal(0, sd ,length1)
    print(" Calculated SNR : = ", snr(Ps, power(noise))[0], " in  dB")
    SNR.append(snr(Ps, power(noise))[0])
    
# Pn = noise_power_for_fixed_SNR_dB(SNR_in_dB, Ps)
# noise = find_noise_for_SNR(length1, Pn, Ps)


    print("Adding noise")
    noise_rep_vectors = representation_vectors + np.reshape(noise, (representation_vectors.shape[0] , representation_vectors.shape[1]))
    #print(" Calculated SNR : = ", snr(Ps, power(noise))[0], " in  dB")
    
    
    try:
        assert np.abs(np.sum(representation_vectors - noise_rep_vectors)) > 0
    except AssertionError:
        logging.error("Noise is NOT added", exc_info=True)

    print(" ")
    print("-----------------------------------------------------")
    print(" Prediction for Test Data")                    
    #test_firingtime = probability_calculation(X_test, timeseries, b)
    y_pred_val = cosine_similar_measure(test_proba, y_test, a, b, c, noise_rep_vectors)

    accuracy = accuracy_score(y_test, y_pred_val)*100
    recall = recall_score(y_test, y_pred_val , average="macro")
    precision = precision_score(y_test, y_pred_val , average="macro")
    f1 = f1_score(y_test, y_pred_val, average="macro")
    ACC[row,0] = accuracy
    PRECISION[row, 0] = precision
    RECALL[row, 0] = recall
    F1SCORE[row, 0] = f1 
        
print("")
print("Saving Results")


path = os.getcwd()
resultpath = path + '/noise-results/'  + data_name + '/'+ method


try:
    os.makedirs(resultpath)
except OSError:
    print ("Creation of the result directory %s failed" % resultpath)
else:
    print ("Successfully created the result directory %s" % resultpath)

np.savetxt(resultpath +"/"+ data_name +"-"+ method +"-ACC.csv",ACC, delimiter= ',', fmt='%1.3f')
np.savetxt(resultpath + "/" + data_name + "-"+ method + "-PRECISION.csv", PRECISION, delimiter= ',', fmt='%1.3f')
np.savetxt(resultpath + "/" + data_name + "-"+ method + "-RECALL.csv", RECALL, delimiter= ',', fmt='%1.3f')
np.savetxt(resultpath + "/" + data_name + "-"+ method + "-F1SCORE.csv", F1SCORE, delimiter= ',', fmt='%1.3f') 
np.savetxt(resultpath + "/" + data_name + "-"+ method + "variance.csv", variance, delimiter= ',', fmt='%1.4f') 
np.savetxt(resultpath + "/" + data_name + "-"+ method + "SNR.csv", SNR, delimiter= ',', fmt='%1.4f') 

print("")
print("Saving Graphs")
import os

# define the name of the directory to be created
path = os.getcwd()
graphpath = path + '/noise-graphs/'  + data_name + '/'+ method

# define the name of the directory to be created


# define the access rights
#access_rights = 0o755

try:
    os.makedirs(graphpath)
except OSError:
    print ("Creation of the result directory %s failed" % graphpath)
else:
    print ("Successfully created the result directory %s" % graphpath)

import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.plot(variance, ACC,linewidth = 2.0)# initial value is 0.0005
plt.xlabel('Standard-deviation', fontsize = 20)
plt.ylabel(' Accuracy', fontsize = 20)
plt.xticks(fontsize=12)
#plt.xlim([0.0, 1.0])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)


plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-ACCURACY.eps", format='eps', dpi=700)

plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-ACCURACY.jpg", format='jpg', dpi=700)

plt.figure(figsize=(10,10))
plt.plot(variance, F1SCORE,linewidth = 2.0)# initial value is 0.0005
plt.xlabel('Standard-deviation', fontsize = 20)
plt.ylabel(' F1-score', fontsize = 20)
plt.xticks(fontsize=12)
#plt.xlim([0.0, 1.0])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)


plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-F1SCORE.eps", format='eps', dpi=700)

plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-F1SCORE.jpg", format='jpg', dpi=700)


plt.figure(figsize=(10,10))
plt.plot(variance, SNR,linewidth = 2.0)# initial value is 0.0005
plt.xlabel('Standard-deviation', fontsize = 20)
plt.ylabel(' SNR', fontsize = 20)
plt.xticks(fontsize=12)
#plt.xlim([0.0, 1.0])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)


plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-SNR.eps", format='eps', dpi=700)

plt.savefig(graphpath +"/"+ data_name +"-"+ method +"-SNR.jpg", format='jpg', dpi=700)


(' Accessing Test Proba ', '/media/harikrishnan/STRONTIUM/TTSS-main-codes/Sept-1-19/chaosnet-results/IRIS/TT-SS/IRIS-test_proba.csv')
 This dataset does not have separate test data
('Number of total data instances', 150)
 ----------Step -1---------------
                                 
 Data normalization done 
('Number of samples_per_class = ', 7)
(' Calculated SNR : = ', 45.36772652586393, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 42.67375504791481, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 40.62050820997186, ' in  dB'

accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 24.397478000240472, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 24.119505809909295, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 23.850154315913148, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
   

accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 18.77422298359371, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 18.62762221006568, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 18.48345475706219, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
      

accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 15.289437899711952, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 15.191013167926553, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 15.093691257462051, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
   

accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 12.735162086450512, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 12.661708036531103, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
       [ 0, 34,  5],
       [ 0,  0, 38]]))
(' Calculated SNR : = ', 12.58886996073717, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
95.833
precision
0.961
recall
0.957
f1score
0.957
('Confusion matrix\n', array([[43,  0,  0],
    

...

accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -5.834755443971771, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -5.843447853641999, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -5.8521315730736685, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
      

accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.09181656193734, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.100255610099918, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.108686466998736, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [

accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.325064727551943, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.33328027759746, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.341488064306082, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [

accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.568212574788306, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.576201436825906, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.584182957845207, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       

accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.804738820408338, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.812513168332204, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       [39,  0,  0],
       [38,  0,  0]]))
(' Calculated SNR : = ', -6.820280564010183, ' in  dB')
Adding noise
 
-----------------------------------------------------
 Prediction for Test Data
accuracy
8.333
precision
0.038
recall
0.078
f1score
0.051
('Confusion matrix\n', array([[10, 33,  0],
       